In [1]:
# this file is used to search for the optimal parameters 
# among all the combination of three kernels and c = [0.01,0.1,1]
# we first determine the best kernel, and then search for the optimal value of c
import numpy as np
from numpy import linalg
from dataloader import *
from svmmodel import *
from datetime import datetime
from sklearn.model_selection import ShuffleSplit

In [2]:
# load mnist dataset
# since we apply cvopt, we need to change data type to double
train_xx, train_yy, testx, testy = loadmnist()
train_x=train_xx.astype(np.double)
train_y=train_yy.astype(np.double)

#apply pac for dimensionn reduction
from sklearn.decomposition import PCA
pca = PCA(n_components=25)
pca.fit(train_x)

# the mnist dataset is too huge to calculate
# we only use 6000 of its trainnig data to do the cross validation
xtrain = pca.transform(train_x[0:6000,:])
ytrain = train_y[0:6000]

In [3]:
#normalization--change all the negative value to positive
mean = xtrain.mean()
var  = xtrain.var()
xtrain= (xtrain - mean)/var
#minvalue = xtrain.min()
#xtrain = xtrain -minvalue

In [4]:
# main function for svmmnist cross validation
# the variable "result" is the value of time running this svm function.
def Runsvmmnist(k,const,trainxdata,trainydata):
    a = datetime.now()
    name=['linear','gaussian','poly_2']
    if k ==0:
        kern = linear_kernel
    if k ==1:
        kern = gaussian_kernel
    if k ==2:
        kern = polynomial_kernel
    rs = ShuffleSplit(n_splits=5, test_size=.2, random_state=0)
    accuracy = 0
    for train_index, test_index in rs.split(trainxdata):
        train_x=trainxdata[train_index]
        train_y=trainydata[train_index]
        test_x=trainxdata[test_index]
        test_y=trainydata[test_index]
# create, train and do prediction with multiple svms for each categories
        final_result = np.zeros(test_y.shape[0])
        count=0
        correct=0
        list = [SVM(C=const,kernel=kern) for i in range(0,10)]
        for i in range(0,10):
            #split dataset into 10 small folders for each category
            traindata  = train_x[count:(i+1)*480,:]
            trainlabel = train_y[count:(i+1)*480] - i + 1

        #for catech dataset, don't need to split the dataset, just use the whole traindata 10 times, so the code should be
        #traindata = train_x
        #trainlabel = train_y - i + 1

            trainlabel=np.array(np.where(trainlabel == 1, trainlabel, -1))
            count=count+480
            list[i].fit(traindata, trainlabel)
            predictions = list[i].predict(test_x)
            final_result=np.array(np.where(predictions !=1, final_result, i))
        correct = correct + np.sum(final_result == test_y)
        correct = float(correct)/(test_y.shape[0])
        accuracy = accuracy + correct
    accuracy = float(accuracy) / 5
    b = datetime.now()
#calculate the accuracy of our prediction
    result = (b-a).seconds
    print("k=",name[k],"c=",const,accuracy,result)
    return accuracy, result

In [5]:
# two for loops is appied for the different combination of two parameters
accuracy = np.zeros(12)
result = np.zeros(12)
count=0
name=['linear','gaussian','poly_2']
c=np.array([0.01,0.1,1])
for k in range(0,3):
    for i in range(0,3):
        accuracy[count], result[count]= Runsvmmnist(k,c[i],xtrain,ytrain)
        count = count +1

     pcost       dcost       gap    pres   dres
 0: -2.6200e+01 -8.4892e+00  3e+03  5e+01  6e-15
 1: -8.9220e-01 -8.4072e+00  4e+01  6e-01  6e-15
 2: -5.6492e-01 -4.7739e+00  6e+00  4e-02  6e-16
 3: -4.9825e-01 -1.1706e+00  8e-01  5e-03  7e-16
 4: -5.5660e-01 -8.9642e-01  4e-01  2e-03  4e-16
 5: -6.0027e-01 -7.4289e-01  2e-01  6e-04  3e-16
 6: -6.2011e-01 -6.8838e-01  7e-02  2e-04  3e-16
 7: -6.3469e-01 -6.5177e-01  2e-02  3e-05  3e-16
 8: -6.3966e-01 -6.4324e-01  4e-03  4e-06  3e-16
 9: -6.4110e-01 -6.4127e-01  2e-04  1e-07  4e-16
10: -6.4117e-01 -6.4118e-01  1e-05  8e-09  4e-16
11: -6.4118e-01 -6.4118e-01  1e-07  8e-11  4e-16
Optimal solution found.
95 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -3.5179e+01 -8.4672e+00  3e+03  5e+01  7e-15
 1: -1.1124e+00 -8.3837e+00  4e+01  6e-01  8e-15
 2: -6.6986e-01 -4.9169e+00  6e+00  5e-02  8e-16
 3: -5.8096e-01 -1.2359e+00  7e-01  4e-03  8e-16
 4: -6.5438e-01 -8.9425e-01  3e-01  1e-03  3e-16
 5: -6.821

     pcost       dcost       gap    pres   dres
 0: -4.9651e+01 -1.0460e+01  4e+03  6e+01  8e-15
 1: -1.4621e+00 -1.0381e+01  6e+01  8e-01  9e-15
 2: -8.5234e-01 -6.6191e+00  8e+00  5e-02  8e-16
 3: -7.8062e-01 -1.7220e+00  1e+00  5e-03  6e-16
 4: -8.4693e-01 -1.2994e+00  5e-01  2e-03  4e-16
 5: -8.9319e-01 -1.1288e+00  2e-01  8e-04  3e-16
 6: -9.3319e-01 -1.0011e+00  7e-02  2e-04  3e-16
 7: -9.4101e-01 -9.8199e-01  4e-02  7e-05  3e-16
 8: -9.5156e-01 -9.6226e-01  1e-02  1e-05  3e-16
 9: -9.5488e-01 -9.5687e-01  2e-03  1e-16  3e-16
10: -9.5577e-01 -9.5586e-01  9e-05  5e-17  4e-16
11: -9.5581e-01 -9.5581e-01  2e-06  5e-17  3e-16
12: -9.5581e-01 -9.5581e-01  3e-08  4e-17  3e-16
Optimal solution found.
119 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -4.0124e+01 -9.5215e+00  3e+03  6e+01  8e-15
 1: -1.2888e+00 -9.4414e+00  5e+01  7e-01  8e-15
 2: -7.7444e-01 -5.8253e+00  7e+00  4e-02  7e-16
 3: -7.2233e-01 -1.6206e+00  1e+00  5e-03  9e-16
 4: -7.91

     pcost       dcost       gap    pres   dres
 0: -4.7282e+01 -9.3182e+00  3e+03  6e+01  9e-15
 1: -1.5569e+00 -9.2367e+00  5e+01  8e-01  8e-15
 2: -8.5568e-01 -5.9065e+00  7e+00  4e-02  7e-16
 3: -7.9217e-01 -1.5430e+00  8e-01  4e-03  6e-16
 4: -8.3916e-01 -1.1121e+00  3e-01  7e-04  3e-16
 5: -8.6183e-01 -1.0286e+00  2e-01  3e-04  2e-16
 6: -8.8597e-01 -9.4534e-01  6e-02  1e-04  2e-16
 7: -8.9767e-01 -9.1827e-01  2e-02  2e-05  2e-16
 8: -9.0251e-01 -9.0914e-01  7e-03  3e-06  3e-16
 9: -9.0483e-01 -9.0566e-01  8e-04  3e-07  3e-16
10: -9.0517e-01 -9.0521e-01  3e-05  7e-09  2e-16
11: -9.0519e-01 -9.0519e-01  1e-06  3e-10  3e-16
12: -9.0519e-01 -9.0519e-01  2e-08  3e-12  3e-16
Optimal solution found.
102 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -5.2909e+01 -1.0005e+01  4e+03  6e+01  9e-15
 1: -1.6057e+00 -9.9234e+00  5e+01  8e-01  9e-15
 2: -8.9493e-01 -6.2245e+00  7e+00  4e-02  7e-16
 3: -8.1968e-01 -1.4015e+00  6e-01  2e-03  1e-15
 4: -8.60

     pcost       dcost       gap    pres   dres
 0: -5.5884e+01 -1.0716e+01  4e+03  6e+01  8e-15
 1: -1.6692e+00 -1.0636e+01  6e+01  8e-01  9e-15
 2: -9.3293e-01 -6.8828e+00  8e+00  4e-02  7e-16
 3: -8.5974e-01 -1.8264e+00  1e+00  5e-03  7e-16
 4: -9.0109e-01 -1.2391e+00  3e-01  1e-03  5e-16
 5: -9.2556e-01 -1.1146e+00  2e-01  4e-04  3e-16
 6: -9.4926e-01 -1.0124e+00  6e-02  1e-04  2e-16
 7: -9.5783e-01 -9.9068e-01  3e-02  3e-05  2e-16
 8: -9.6497e-01 -9.7522e-01  1e-02  6e-06  2e-16
 9: -9.6765e-01 -9.7043e-01  3e-03  6e-07  3e-16
10: -9.6835e-01 -9.6931e-01  1e-03  4e-08  2e-16
11: -9.6874e-01 -9.6885e-01  1e-04  4e-09  2e-16
12: -9.6878e-01 -9.6879e-01  1e-05  5e-11  3e-16
13: -9.6879e-01 -9.6879e-01  3e-07  1e-12  3e-16
Optimal solution found.
112 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -3.6176e+01 -9.5908e+00  3e+03  6e+01  7e-15
 1: -1.2024e+00 -9.5086e+00  5e+01  8e-01  7e-15
 2: -7.0861e-01 -5.9780e+00  7e+00  5e-02  7e-16
 3: -6.35

16: -6.3602e-01 -6.3602e-01  1e-08  5e-17  2e-16
Optimal solution found.
74 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -3.5111e+01 -8.9861e+00  3e+03  5e+01  7e-15
 1: -1.2174e+00 -8.9043e+00  5e+01  7e-01  7e-15
 2: -7.2915e-01 -5.5298e+00  7e+00  5e-02  7e-16
 3: -6.7341e-01 -1.5467e+00  1e+00  6e-03  8e-16
 4: -7.6014e-01 -1.0667e+00  3e-01  1e-03  4e-16
 5: -7.8792e-01 -9.7585e-01  2e-01  7e-04  3e-16
 6: -8.1581e-01 -8.9976e-01  9e-02  2e-04  3e-16
 7: -8.3175e-01 -8.6336e-01  3e-02  7e-05  3e-16
 8: -8.4041e-01 -8.4711e-01  7e-03  1e-05  3e-16
 9: -8.4273e-01 -8.4333e-01  6e-04  8e-07  3e-16
10: -8.4296e-01 -8.4300e-01  4e-05  5e-08  3e-16
11: -8.4297e-01 -8.4298e-01  3e-06  3e-09  3e-16
12: -8.4298e-01 -8.4298e-01  9e-08  8e-11  3e-16
Optimal solution found.
119 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -3.8467e+01 -9.8535e+00  4e+03  6e+01  8e-15
 1: -1.2275e+00 -9.7738e+00  5e+01  7e-01  8e-

...

     pcost       dcost       gap    pres   dres
 0: -1.7629e+00 -5.7645e+01  1e+03  2e+01  1e-14
 1: -5.9103e-01 -4.9599e+01  1e+02  8e-01  1e-14
 2: -2.3137e-01 -1.4029e+01  2e+01  9e-02  3e-15
 3: -7.5219e-02 -2.1721e+00  3e+00  1e-02  2e-15
 4: -6.8980e-02 -1.8309e-01  1e-01  2e-04  7e-16
 5: -8.6626e-02 -1.2772e-01  4e-02  6e-05  5e-16
 6: -9.7425e-02 -1.0810e-01  1e-02  5e-06  5e-16
 7: -1.0066e-01 -1.0345e-01  3e-03  2e-16  5e-16
 8: -1.0161e-01 -1.0190e-01  3e-04  2e-16  5e-16
 9: -1.0173e-01 -1.0174e-01  5e-06  2e-16  4e-16
10: -1.0173e-01 -1.0173e-01  8e-08  2e-16  4e-16
Optimal solution found.
55 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -3.7735e+00 -5.7884e+01  1e+03  2e+01  1e-14
 1: -1.2290e+00 -4.9791e+01  1e+02  7e-01  1e-14
 2: -6.0773e-01 -1.3364e+01  2e+01  9e-02  4e-15
 3: -3.1045e-01 -2.6993e+00  3e+00  1e-02  2e-15
 4: -2.4070e-01 -4.3358e-01  2e-01  6e-04  1e-15
 5: -2.6575e-01 -3.0673e-01  4e-02  9e-05  9e-16
 6: -2.749

     pcost       dcost       gap    pres   dres
 0: -8.7762e+00 -6.9648e+02  3e+03  2e+00  4e-14
 1: -4.4251e+00 -3.1403e+02  6e+02  3e-01  3e-14
 2: -5.9941e-01 -5.4803e+01  8e+01  3e-02  2e-14
 3: -8.2527e-02 -8.5054e+00  1e+01  4e-03  5e-15
 4: -2.2406e-01 -1.3667e+00  1e+00  3e-04  2e-15
 5: -3.5277e-01 -6.0103e-01  3e-01  5e-05  1e-15
 6: -4.0210e-01 -4.7575e-01  7e-02  1e-06  1e-15
 7: -4.2079e-01 -4.3794e-01  2e-02  7e-08  1e-15
 8: -4.2554e-01 -4.2934e-01  4e-03  1e-08  1e-15
 9: -4.2685e-01 -4.2711e-01  3e-04  4e-12  1e-15
10: -4.2695e-01 -4.2696e-01  5e-06  6e-14  1e-15
11: -4.2696e-01 -4.2696e-01  1e-07  9e-16  1e-15
Optimal solution found.
82 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -9.2471e+00 -6.7605e+02  3e+03  2e+00  4e-14
 1: -4.3970e+00 -2.8748e+02  5e+02  2e-01  3e-14
 2: -9.8662e-01 -7.4744e+01  1e+02  4e-02  2e-14
 3: -3.0501e-01 -1.5098e+01  2e+01  7e-03  8e-15
 4: -3.5259e-01 -2.0167e+00  2e+00  3e-04  4e-15
 5: -5.487

     pcost       dcost       gap    pres   dres
 0: -5.8152e+00 -6.7026e+02  3e+03  2e+00  3e-14
 1: -2.4456e+00 -2.8454e+02  5e+02  2e-01  2e-14
 2: -1.1544e-01 -4.4955e+01  7e+01  2e-02  2e-14
 3: -4.2384e-02 -8.0375e+00  1e+01  3e-03  4e-15
 4: -2.3766e-01 -8.1714e-01  6e-01  3e-05  2e-15
 5: -3.5808e-01 -5.1008e-01  2e-01  6e-06  1e-15
 6: -3.9574e-01 -4.5042e-01  6e-02  2e-06  1e-15
 7: -4.1346e-01 -4.2986e-01  2e-02  1e-07  1e-15
 8: -4.1893e-01 -4.2093e-01  2e-03  2e-16  1e-15
 9: -4.1968e-01 -4.1982e-01  1e-04  2e-16  1e-15
10: -4.1974e-01 -4.1974e-01  3e-06  2e-16  1e-15
11: -4.1974e-01 -4.1974e-01  8e-08  2e-16  1e-15
Optimal solution found.
84 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -6.5321e+00 -6.7323e+02  3e+03  2e+00  3e-14
 1: -3.0751e+00 -2.8146e+02  5e+02  2e-01  2e-14
 2: -3.5765e-01 -4.6637e+01  7e+01  3e-02  1e-14
 3: -3.9513e-02 -5.1440e+00  7e+00  2e-03  3e-15
 4: -1.3741e-01 -5.0693e-01  4e-01  1e-05  1e-15
 5: -2.255

     pcost       dcost       gap    pres   dres
 0: -7.3921e+00 -7.0190e+02  3e+03  2e+00  3e-14
 1: -3.8811e+00 -3.2454e+02  6e+02  3e-01  3e-14
 2: -6.3558e-01 -6.0776e+01  9e+01  4e-02  2e-14
 3: -1.9014e-01 -9.7745e+00  1e+01  5e-03  5e-15
 4: -2.2020e-01 -1.5053e+00  2e+00  3e-04  2e-15
 5: -3.4030e-01 -6.1504e-01  3e-01  6e-05  1e-15
 6: -3.8377e-01 -4.7601e-01  1e-01  1e-05  1e-15
 7: -4.0089e-01 -4.4190e-01  4e-02  8e-07  1e-15
 8: -4.1110e-01 -4.2077e-01  1e-02  2e-07  1e-15
 9: -4.1423e-01 -4.1519e-01  1e-03  2e-16  1e-15
10: -4.1462e-01 -4.1465e-01  3e-05  2e-16  2e-15
11: -4.1464e-01 -4.1464e-01  6e-07  2e-16  1e-15
12: -4.1464e-01 -4.1464e-01  1e-08  2e-16  1e-15
Optimal solution found.
63 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -1.2374e+01 -7.1571e+02  3e+03  2e+00  4e-14
 1: -6.2771e+00 -3.3372e+02  6e+02  3e-01  3e-14
 2: -9.3037e-01 -7.1344e+01  1e+02  4e-02  2e-14
 3: -2.8883e-01 -1.2753e+01  2e+01  6e-03  7e-15
 4: -4.351

     pcost       dcost       gap    pres   dres
 0: -1.1930e+01 -6.7182e+02  3e+03  2e+00  5e-14
 1: -6.2975e+00 -3.1482e+02  6e+02  3e-01  4e-14
 2: -1.1652e+00 -9.8836e+01  2e+02  7e-02  2e-14
 3: -3.4202e-01 -1.7146e+01  3e+01  9e-03  9e-15
 4: -3.5287e-01 -2.8577e+00  3e+00  7e-04  4e-15
 5: -5.5909e-01 -9.3104e-01  4e-01  7e-05  2e-15
 6: -6.3935e-01 -7.4036e-01  1e-01  1e-05  2e-15
 7: -6.6766e-01 -6.9327e-01  3e-02  2e-06  2e-15
 8: -6.7615e-01 -6.8045e-01  4e-03  2e-16  2e-15
 9: -6.7773e-01 -6.7794e-01  2e-04  2e-16  2e-15
10: -6.7782e-01 -6.7782e-01  5e-06  2e-16  2e-15
11: -6.7782e-01 -6.7782e-01  1e-07  2e-16  2e-15
Optimal solution found.
93 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -2.7609e+00 -6.5092e+02  3e+03  2e+00  2e-14
 1: -1.2618e+00 -2.7317e+02  5e+02  2e-01  2e-14
 2: -1.4726e-02 -2.9651e+01  4e+01  2e-02  1e-14
 3:  3.2113e-02 -3.6516e+00  5e+00  2e-03  3e-15
 4: -2.9728e-02 -2.6501e-01  2e-01  2e-05  1e-15
 5: -7.558

In [6]:
# show the final result 
count = 0
for k in range(0,3):
    for i in range(0,3):
        print("k=",name[k],"c=",c[i],accuracy[count],result[count])
        count = count +1

k= linear c= 0.01 0.659833333333 23.0
k= linear c= 0.1 0.7635 20.0
k= linear c= 1.0 0.757333333333 20.0
k= gaussian c= 0.01 0.0968333333333 121.0
k= gaussian c= 0.1 0.455166666667 119.0
k= gaussian c= 1.0 0.770166666667 112.0
k= poly_2 c= 0.01 0.8395 31.0
k= poly_2 c= 0.1 0.8415 29.0
k= poly_2 c= 1.0 0.838833333333 30.0


In [8]:
# after we find the optimal parameters which is poly_kernel and c=0.03
# we do cross validation again to find the best c when kernel is poly kernel
c1=np.array([0.03,0.05,0.07,0.09,0.1,0.3,0.5,0.7,0.9])
accuracy1 = np.zeros(9)
result1 = np.zeros(9)
count = 0
for i in range(0,9):        
    accuracy1[count], result1[count]= Runsvmmnist(2,c1[i],xtrain,ytrain)
    count = count +1

     pcost       dcost       gap    pres   dres
 0: -1.4716e+00 -1.7699e+01  1e+03  4e+01  1e-14
 1: -3.3627e-01 -1.6841e+01  5e+01  8e-01  1e-14
 2: -1.8434e-01 -6.3276e+00  9e+00  1e-01  3e-15
 3: -7.6934e-02 -1.0055e+00  1e+00  1e-02  1e-15
 4: -7.7713e-02 -2.0559e-01  1e-01  1e-03  7e-16
 5: -8.8220e-02 -1.3688e-01  5e-02  4e-04  4e-16
 6: -9.5143e-02 -1.1064e-01  2e-02  6e-05  4e-16
 7: -9.8601e-02 -1.0229e-01  4e-03  1e-16  4e-16
 8: -9.9874e-02 -1.0021e-01  3e-04  1e-16  6e-16
 9: -1.0001e-01 -1.0002e-01  9e-06  1e-16  5e-16
10: -1.0002e-01 -1.0002e-01  2e-07  1e-16  6e-16
11: -1.0002e-01 -1.0002e-01  3e-09  1e-16  6e-16
Optimal solution found.
57 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -4.0011e+00 -1.9334e+01  2e+03  4e+01  1e-14
 1: -8.7942e-01 -1.8325e+01  7e+01  1e+00  1e-14
 2: -5.1427e-01 -8.0231e+00  1e+01  1e-01  3e-15
 3: -3.3954e-01 -1.8137e+00  2e+00  2e-02  1e-15
 4: -2.8700e-01 -5.3422e-01  3e-01  3e-03  1e-15
 5: -3.000

     pcost       dcost       gap    pres   dres
 0: -5.0151e+00 -2.1255e+01  2e+03  4e+01  2e-14
 1: -1.1605e+00 -2.0231e+01  1e+02  2e+00  2e-14
 2: -6.6218e-01 -1.1460e+01  2e+01  2e-01  3e-15
 3: -4.0903e-01 -4.1133e+00  5e+00  5e-02  2e-15
 4: -3.5191e-01 -1.1040e+00  9e-01  8e-03  9e-16
 5: -3.9290e-01 -5.8329e-01  2e-01  1e-03  9e-16
 6: -4.1646e-01 -4.9158e-01  8e-02  5e-04  9e-16
 7: -4.3074e-01 -4.4499e-01  1e-02  5e-06  1e-15
 8: -4.3556e-01 -4.3634e-01  8e-04  7e-08  1e-15
 9: -4.3587e-01 -4.3589e-01  2e-05  1e-09  1e-15
10: -4.3588e-01 -4.3588e-01  5e-07  2e-11  1e-15
11: -4.3588e-01 -4.3588e-01  2e-08  2e-13  1e-15
Optimal solution found.
92 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -5.0448e+00 -1.9501e+01  2e+03  4e+01  2e-14
 1: -1.2026e+00 -1.8459e+01  7e+01  1e+00  2e-14
 2: -7.1176e-01 -9.2218e+00  1e+01  2e-01  4e-15
 3: -4.8703e-01 -2.8103e+00  3e+00  3e-02  2e-15
 4: -4.7655e-01 -9.3308e-01  5e-01  5e-03  1e-15
 5: -5.076

     pcost       dcost       gap    pres   dres
 0: -3.4632e+00 -1.9477e+01  2e+03  4e+01  2e-14
 1: -7.2652e-01 -1.8562e+01  6e+01  1e+00  2e-14
 2: -4.1556e-01 -8.3978e+00  1e+01  2e-01  3e-15
 3: -2.3236e-01 -1.8299e+00  2e+00  2e-02  1e-15
 4: -2.3974e-01 -5.7768e-01  4e-01  3e-03  8e-16
 5: -2.6720e-01 -3.5970e-01  1e-01  7e-04  8e-16
 6: -2.8273e-01 -3.0241e-01  2e-02  9e-05  9e-16
 7: -2.8752e-01 -2.9167e-01  4e-03  8e-06  8e-16
 8: -2.8893e-01 -2.8924e-01  3e-04  2e-07  9e-16
 9: -2.8906e-01 -2.8906e-01  7e-06  3e-09  9e-16
10: -2.8906e-01 -2.8906e-01  1e-07  3e-11  1e-15
Optimal solution found.
75 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -4.4186e+00 -1.8635e+01  2e+03  4e+01  2e-14
 1: -1.0495e+00 -1.7614e+01  9e+01  2e+00  2e-14
 2: -6.0480e-01 -1.0238e+01  2e+01  3e-01  3e-15
 3: -3.6404e-01 -2.7321e+00  3e+00  4e-02  1e-15
 4: -3.1302e-01 -7.2384e-01  5e-01  5e-03  9e-16
 5: -3.3936e-01 -4.3840e-01  1e-01  6e-04  8e-16
 6: -3.541

     pcost       dcost       gap    pres   dres
 0: -7.0652e+00 -2.2123e+01  2e+03  4e+01  3e-14
 1: -1.5122e+00 -2.1009e+01  9e+01  2e+00  2e-14
 2: -8.6443e-01 -1.2133e+01  2e+01  2e-01  5e-15
 3: -5.6892e-01 -3.3719e+00  4e+00  4e-02  2e-15
 4: -5.4145e-01 -1.0907e+00  7e-01  6e-03  2e-15
 5: -5.8810e-01 -7.2461e-01  2e-01  1e-03  1e-15
 6: -6.1091e-01 -6.4221e-01  3e-02  2e-04  1e-15
 7: -6.1807e-01 -6.2370e-01  6e-03  2e-05  1e-15
 8: -6.1994e-01 -6.2014e-01  2e-04  2e-07  2e-15
 9: -6.2002e-01 -6.2002e-01  5e-06  3e-09  2e-15
10: -6.2002e-01 -6.2002e-01  2e-07  6e-11  2e-15
Optimal solution found.
102 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -1.6685e+00 -1.7287e+01  1e+03  4e+01  1e-14
 1: -3.8987e-01 -1.6410e+01  6e+01  1e+00  1e-14
 2: -2.0803e-01 -7.4808e+00  1e+01  1e-01  2e-15
 3: -9.5823e-02 -1.6913e+00  2e+00  3e-02  9e-16
 4: -6.9871e-02 -2.0635e-01  1e-01  7e-04  7e-16
 5: -8.6245e-02 -1.3727e-01  5e-02  2e-04  5e-16
 6: -9.72

     pcost       dcost       gap    pres   dres
 0: -5.0861e+00 -3.5849e+01  2e+03  4e+01  2e-14
 1: -1.0847e+00 -3.3585e+01  1e+02  1e+00  2e-14
 2: -5.8659e-01 -1.3083e+01  2e+01  2e-01  4e-15
 3: -2.9450e-01 -2.7775e+00  3e+00  2e-02  2e-15
 4: -3.3256e-01 -6.9555e-01  4e-01  2e-03  1e-15
 5: -3.7278e-01 -4.7994e-01  1e-01  4e-04  8e-16
 6: -3.9396e-01 -4.1981e-01  3e-02  4e-05  1e-15
 7: -4.0095e-01 -4.0450e-01  4e-03  4e-07  1e-15
 8: -4.0223e-01 -4.0233e-01  1e-04  9e-09  1e-15
 9: -4.0227e-01 -4.0227e-01  2e-06  1e-10  1e-15
10: -4.0227e-01 -4.0227e-01  2e-08  1e-12  1e-15
Optimal solution found.
101 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -5.2452e+00 -3.1154e+01  2e+03  4e+01  2e-14
 1: -1.2531e+00 -2.8836e+01  8e+01  1e+00  2e-14
 2: -7.1527e-01 -1.1114e+01  2e+01  2e-01  4e-15
 3: -3.7712e-01 -3.1977e+00  4e+00  4e-02  2e-15
 4: -3.5087e-01 -6.5252e-01  3e-01  2e-03  1e-15
 5: -3.9119e-01 -4.9287e-01  1e-01  5e-04  1e-15
 6: -4.13

...

     pcost       dcost       gap    pres   dres
 0: -7.9233e+00 -4.3973e+02  2e+03  3e+00  3e-14
 1: -4.1699e+00 -2.1977e+02  4e+02  3e-01  3e-14
 2: -1.0304e+00 -5.0667e+01  7e+01  5e-02  2e-14
 3: -3.4823e-01 -7.1657e+00  9e+00  5e-03  5e-15
 4: -3.1770e-01 -1.5934e+00  1e+00  5e-04  2e-15
 5: -4.1022e-01 -7.5941e-01  4e-01  1e-04  1e-15
 6: -4.5788e-01 -5.4635e-01  9e-02  2e-05  1e-15
 7: -4.7758e-01 -4.9818e-01  2e-02  3e-06  1e-15
 8: -4.8376e-01 -4.8609e-01  2e-03  2e-07  1e-15
 9: -4.8461e-01 -4.8469e-01  8e-05  6e-09  1e-15
10: -4.8464e-01 -4.8465e-01  3e-06  2e-10  2e-15
11: -4.8465e-01 -4.8465e-01  8e-08  2e-12  2e-15
Optimal solution found.
81 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -7.0896e+00 -4.3819e+02  2e+03  3e+00  3e-14
 1: -3.6017e+00 -2.1832e+02  4e+02  3e-01  3e-14
 2: -4.6270e-01 -4.3662e+01  6e+01  4e-02  1e-14
 3: -2.3297e-01 -6.9182e+00  9e+00  4e-03  4e-15
 4: -3.1372e-01 -8.9378e-01  6e-01  1e-04  2e-15
 5: -4.064

     pcost       dcost       gap    pres   dres
 0: -1.3468e+01 -6.1815e+02  3e+03  2e+00  4e-14
 1: -7.4365e+00 -2.9604e+02  5e+02  3e-01  4e-14
 2: -1.8309e+00 -7.1939e+01  1e+02  5e-02  2e-14
 3: -7.9650e-01 -1.4469e+01  2e+01  8e-03  7e-15
 4: -7.2305e-01 -4.0112e+00  4e+00  1e-03  4e-15
 5: -8.2671e-01 -1.3656e+00  6e-01  2e-04  2e-15
 6: -9.1761e-01 -1.0372e+00  1e-01  3e-05  2e-15
 7: -9.4953e-01 -9.7031e-01  2e-02  2e-06  2e-15
 8: -9.5701e-01 -9.5798e-01  1e-03  4e-08  2e-15
 9: -9.5742e-01 -9.5744e-01  2e-05  8e-10  2e-15
10: -9.5743e-01 -9.5743e-01  1e-06  1e-11  2e-15
11: -9.5743e-01 -9.5743e-01  7e-08  2e-13  2e-15
Optimal solution found.
85 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -4.2324e+00 -5.5120e+02  3e+03  2e+00  3e-14
 1: -1.9673e+00 -2.3022e+02  4e+02  2e-01  3e-14
 2: -3.1658e-01 -4.2586e+01  6e+01  3e-02  1e-14
 3: -5.5628e-02 -5.4475e+00  8e+00  3e-03  4e-15
 4: -5.7156e-02 -3.7884e-01  3e-01  3e-05  1e-15
 5: -1.168

     pcost       dcost       gap    pres   dres
 0: -7.2128e+00 -5.5935e+02  3e+03  2e+00  4e-14
 1: -3.6504e+00 -2.3988e+02  4e+02  2e-01  2e-14
 2: -7.2353e-01 -4.4214e+01  7e+01  3e-02  1e-14
 3: -2.6204e-01 -6.7672e+00  9e+00  4e-03  4e-15
 4: -1.8323e-01 -1.2705e+00  1e+00  4e-04  2e-15
 5: -2.3938e-01 -4.6028e-01  3e-01  7e-05  1e-15
 6: -2.7231e-01 -3.3699e-01  7e-02  2e-05  1e-15
 7: -2.9036e-01 -2.9699e-01  7e-03  2e-16  1e-15
 8: -2.9309e-01 -2.9363e-01  5e-04  2e-16  1e-15
 9: -2.9334e-01 -2.9335e-01  1e-05  2e-16  1e-15
10: -2.9334e-01 -2.9334e-01  2e-07  2e-16  1e-15
Optimal solution found.
47 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -6.4315e+00 -5.6098e+02  3e+03  2e+00  3e-14
 1: -3.0609e+00 -2.4257e+02  4e+02  2e-01  3e-14
 2: -4.5390e-01 -3.8908e+01  6e+01  3e-02  1e-14
 3: -1.0144e-01 -5.9106e+00  8e+00  3e-03  4e-15
 4: -2.4091e-01 -8.9318e-01  7e-01  1e-04  2e-15
 5: -3.2436e-01 -5.2215e-01  2e-01  3e-05  1e-15
 6: -3.552

     pcost       dcost       gap    pres   dres
 0: -5.8438e+00 -5.5975e+02  3e+03  2e+00  3e-14
 1: -2.6485e+00 -2.4364e+02  4e+02  2e-01  2e-14
 2: -1.7731e-01 -3.7496e+01  5e+01  2e-02  1e-14
 3: -1.0698e-01 -6.0100e+00  8e+00  3e-03  3e-15
 4: -2.0943e-01 -1.3183e+00  1e+00  3e-04  1e-15
 5: -3.0097e-01 -5.2751e-01  2e-01  6e-05  1e-15
 6: -3.3522e-01 -4.0579e-01  7e-02  1e-05  8e-16
 7: -3.5055e-01 -3.7147e-01  2e-02  2e-16  1e-15
 8: -3.5670e-01 -3.5903e-01  2e-03  2e-16  1e-15
 9: -3.5754e-01 -3.5762e-01  8e-05  2e-16  9e-16
10: -3.5757e-01 -3.5757e-01  1e-06  2e-16  1e-15
11: -3.5757e-01 -3.5757e-01  1e-08  2e-16  9e-16
Optimal solution found.
86 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -6.2573e+00 -5.5336e+02  3e+03  2e+00  3e-14
 1: -3.0742e+00 -2.3637e+02  4e+02  2e-01  2e-14
 2: -4.8964e-01 -4.3104e+01  6e+01  3e-02  1e-14
 3: -1.9928e-01 -7.6644e+00  1e+01  4e-03  4e-15
 4: -3.0752e-01 -7.9918e-01  5e-01  4e-05  2e-15
 5: -4.108

     pcost       dcost       gap    pres   dres
 0: -4.2998e+00 -5.4052e+02  3e+03  2e+00  2e-14
 1: -1.9143e+00 -2.2314e+02  4e+02  2e-01  2e-14
 2: -1.9652e-02 -3.6299e+01  5e+01  2e-02  1e-14
 3: -4.3453e-02 -5.5663e+00  7e+00  2e-03  3e-15
 4: -2.1034e-01 -5.9377e-01  4e-01  1e-05  1e-15
 5: -2.8538e-01 -3.9434e-01  1e-01  4e-06  9e-16
 6: -3.1496e-01 -3.5071e-01  4e-02  6e-07  8e-16
 7: -3.2368e-01 -3.3821e-01  1e-02  2e-07  8e-16
 8: -3.2784e-01 -3.3276e-01  5e-03  1e-16  1e-15
 9: -3.2949e-01 -3.2984e-01  4e-04  2e-16  9e-16
10: -3.2962e-01 -3.2964e-01  2e-05  2e-16  1e-15
11: -3.2963e-01 -3.2963e-01  4e-07  2e-16  1e-15
12: -3.2963e-01 -3.2963e-01  8e-09  2e-16  1e-15
Optimal solution found.
88 support vectors out of 480 points
     pcost       dcost       gap    pres   dres
 0: -4.6402e+00 -6.1082e+02  3e+03  2e+00  3e-14
 1: -2.3421e+00 -2.8917e+02  5e+02  3e-01  2e-14
 2: -1.9344e-01 -4.7192e+01  7e+01  3e-02  1e-14
 3: -3.5757e-02 -6.6911e+00  9e+00  4e-03  3e-15
 4: -9.063

In [10]:
# show the final result
count = 0
for i in range(0,9):
    print("k=",name[2],"c=",c1[i],accuracy1[count],result1[count])
    count = count +1

k= poly_2 c= 0.03 0.848666666667 30.0
k= poly_2 c= 0.05 0.844833333333 30.0
k= poly_2 c= 0.07 0.844166666667 30.0
k= poly_2 c= 0.09 0.842 30.0
k= poly_2 c= 0.1 0.8415 30.0
k= poly_2 c= 0.3 0.838833333333 30.0
k= poly_2 c= 0.5 0.838833333333 30.0
k= poly_2 c= 0.7 0.838833333333 30.0
k= poly_2 c= 0.9 0.838833333333 30.0
